In [ ]:
import pandas as pd
import requests
import zipfile
import io
import os
import csv
import ctypes as ct
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
from matplotlib.dates import YearLocator
from matplotlib.ticker import PercentFormatter
import re
import seaborn as sns
import QuantLib as ql

from bgs.bgs_utils import clean_date, clean_percentage
from bgs.load_bgs_amounts import load_bgs_amounts
from bgs.load_gilt_details import load_csv_blocks
from bgs.load_bgs_prices import load_prices
from bgs.gilt_analytics import gilt_yield, yield_series, yield_curve, from_iso, yield_curves_pw, yield_curve_rv, yield_curve_pw, Gilt
from bgs.linker_analytics import linker_real_yield

from sklearn.pipeline import make_pipeline
from sklearn import preprocessing
from sklearn.svm import SVR

%load_ext autoreload
%autoreload 2


pipeline = make_pipeline(
    preprocessing.StandardScaler(),
    SVR(kernel='linear', epsilon=0.01, C=100, gamma = 0.01),
)

### British Gilt Survey Amounts Table

In [ ]:
tables = load_bgs_amounts("downloads/BGSAmounts.csv")

conv = tables['Conventionals']
conv.replace("Redeemed", "", inplace=True)
conv = conv.apply(pd.to_numeric, errors='coerce')
conv.fillna(0, inplace=True)
conv.index = pd.Index([clean_date(x) for x in list(conv.index)]).to_period('M').to_timestamp('M')
conv.columns = [np.int64(x.strip()) for x in conv.columns]

old = tables['Calculated indexed nominal Old-style']
old.replace("Redeemed", "", inplace=True)
old = old.apply(pd.to_numeric, errors='coerce')
old.fillna(0, inplace=True)
old.index = pd.Index([clean_date(x) for x in list(old.index)]).to_period('M').to_timestamp('M')
old.columns = [np.int64(x.strip()) for x in old.columns]

new = tables['Calculated indexed nominal New-style']
new.replace("Redeemed", "", inplace=True)
new = new.apply(pd.to_numeric, errors='coerce')
new.fillna(0, inplace=True)
new.index = pd.Index([clean_date(x) for x in list(new.index)]).to_period('M').to_timestamp('M')
new.columns = [np.int64(x.strip()) for x in new.columns]

new_no_idx = tables['Index-linked New-style']
new_no_idx.replace("Redeemed", "", inplace=True)
new_no_idx = new_no_idx.apply(pd.to_numeric, errors='coerce')
new_no_idx.fillna(0, inplace=True)
new_no_idx.index = pd.Index([clean_date(x) for x in list(new_no_idx.index)]).to_period('M').to_timestamp('M')
new_no_idx.columns = [np.int64(x.strip()) for x in new_no_idx.columns]

### British Gilts Survey Month End Prices

In [ ]:
price_df=load_prices("downloads/BGSPrices.csv")
price_df.index = pd.to_datetime(price_df.index, format="%d %b %Y").to_period('M').to_timestamp('M')
for text in ['Amalgamated', 'Redeemed', 'redeemed']:
    price_df = price_df.replace(text, 0)
price_df = price_df.replace('missing', None)
price_df = price_df.ffill(axis=0)
price_df = price_df.fillna(0)
price_df.columns = [np.int64(x) for x in price_df.columns]

### British Gilts Survey Details Table (Bond Static Data, Coupon Maturity etc.) 

In [ ]:
details = load_csv_blocks("downloads/BGSDetails.csv")

conv_details = details['Conventionals']
conv_details['%'] = conv_details['%'].apply(clean_percentage)
conv_details['Sequence'] = conv_details['Sequence'].apply(lambda x: np.int64(x))

new_details = details['Index-Linked New-style']
new_details['%'] = new_details['%'].apply(clean_percentage)
new_details['Sequence'] = new_details['Sequence'].apply(lambda x: np.int64(x))

old_details = details['Index-Linked Old-style']
old_details['%'] = old_details['%'].apply(clean_percentage)
old_details['Sequence'] = old_details['Sequence'].apply(lambda x: np.int64(x))

strip_details = details['Strips']
strip_details['%'] = 0.0
strip_details['Sequence'] = strip_details['Sequence'].apply(lambda x: np.int64(x))

date_variables = ["Latest redemption date","Issue date","First coupon payable on date",]
for date_var in date_variables:
    conv_details[date_var] = conv_details[date_var].apply(clean_date)
    old_details[date_var] = old_details[date_var].apply(clean_date)
    new_details[date_var] = new_details[date_var].apply(clean_date)
strip_details['Latest redemption date'] = strip_details['Latest redemption date'].apply(clean_date)

### British Gilt Survey Amounts Table

In [ ]:
tables = load_bgs_amounts("downloads/BGSAmounts.csv")

conv = tables['Conventionals']
conv.replace("Redeemed", "", inplace=True)
conv = conv.apply(pd.to_numeric, errors='coerce')
conv.fillna(0, inplace=True)
conv.index = pd.Index([clean_date(x) for x in list(conv.index)]).to_period('M').to_timestamp('M')
conv.columns = [np.int64(x.strip()) for x in conv.columns]

old = tables['Calculated indexed nominal Old-style']
old.replace("Redeemed", "", inplace=True)
old = old.apply(pd.to_numeric, errors='coerce')
old.fillna(0, inplace=True)
old.index = pd.Index([clean_date(x) for x in list(old.index)]).to_period('M').to_timestamp('M')
old.columns = [np.int64(x.strip()) for x in old.columns]

new = tables['Calculated indexed nominal New-style']
new.replace("Redeemed", "", inplace=True)
new = new.apply(pd.to_numeric, errors='coerce')
new.fillna(0, inplace=True)
new.index = pd.Index([clean_date(x) for x in list(new.index)]).to_period('M').to_timestamp('M')
new.columns = [np.int64(x.strip()) for x in new.columns]

new_no_idx = tables['Index-linked New-style']
new_no_idx.replace("Redeemed", "", inplace=True)
new_no_idx = new_no_idx.apply(pd.to_numeric, errors='coerce')
new_no_idx.fillna(0, inplace=True)
new_no_idx.index = pd.Index([clean_date(x) for x in list(new_no_idx.index)]).to_period('M').to_timestamp('M')
new_no_idx.columns = [np.int64(x.strip()) for x in new_no_idx.columns]

### British Gilts Survey Month End Prices

In [ ]:
price_df=load_prices("downloads/BGSPrices.csv")
price_df.index = pd.to_datetime(price_df.index, format="%d %b %Y").to_period('M').to_timestamp('M')
for text in ['Amalgamated', 'Redeemed', 'redeemed']:
    price_df = price_df.replace(text, 0)
price_df = price_df.replace('missing', None)
price_df = price_df.ffill(axis=0)
price_df = price_df.fillna(0)
price_df.columns = [np.int64(x) for x in price_df.columns]

### British Gilts Survey Details Table (Bond Static Data, Coupon Maturity etc.) 

In [ ]:
details = load_csv_blocks("downloads/BGSDetails.csv")

conv_details = details['Conventionals']
conv_details['%'] = conv_details['%'].apply(clean_percentage)
conv_details['Sequence'] = conv_details['Sequence'].apply(lambda x: np.int64(x))

new_details = details['Index-Linked New-style']
new_details['%'] = new_details['%'].apply(clean_percentage)
new_details['Sequence'] = new_details['Sequence'].apply(lambda x: np.int64(x))

old_details = details['Index-Linked Old-style']
old_details['%'] = old_details['%'].apply(clean_percentage)
old_details['Sequence'] = old_details['Sequence'].apply(lambda x: np.int64(x))

strip_details = details['Strips']
strip_details['%'] = 0.0
strip_details['Sequence'] = strip_details['Sequence'].apply(lambda x: np.int64(x))

date_variables = ["Latest redemption date","Issue date","First coupon payable on date",]
for date_var in date_variables:
    conv_details[date_var] = conv_details[date_var].apply(clean_date)
    old_details[date_var] = old_details[date_var].apply(clean_date)
    new_details[date_var] = new_details[date_var].apply(clean_date)
strip_details['Latest redemption date'] = strip_details['Latest redemption date'].apply(clean_date)

In [ ]:
all_conv = conv_details[['Sequence', 'ISIN Code','%','Latest redemption date', 'Issue date', 'First coupon payable on date']]


In [ ]:
month_end_dates = pd.date_range(start='2024-01-31', end='2025-04-30', freq='ME').strftime("%Y-%m-%d").to_list()

In [ ]:
def bgs_gilt_yield(bgs_id, calc_date):

    bond = conv_details[conv_details['Sequence']==bgs_id]
    issue_date=bond['Issue date'].dt.strftime("%Y-%m-%d").values[0]
    maturity_date=bond['Latest redemption date'].dt.strftime("%Y-%m-%d").values[0]
    first_cpn_date=bond['First coupon payable on date'].dt.strftime("%Y-%m-%d").values[0]
    last_cpn_date=bond['Latest redemption date'].dt.strftime("%Y-%m-%d").values[0]
    coupon=bond['%'].values[0]/100

    clean_price=price_df.loc[calc_date,bgs_id]


    return gilt_yield(
        trade_date=calc_date,
        issue_date=issue_date,
        maturity_date=maturity_date,
        first_cpn_date=first_cpn_date,
        last_cpn_date=last_cpn_date,
        clean_price=clean_price,
        coupon=coupon)

In [ ]:
dislocations = []
starting = all_conv.copy()
calculation_date = month_end_dates[0]

gilts_list = starting.loc[(starting['Latest redemption date'] > pd.to_datetime(calculation_date))&(starting['Issue date']< pd.to_datetime(calculation_date))]
gilts_list['Amount_Outstanding']=gilts_list['Sequence'].map( lambda x: conv.loc[calculation_date, x])
gilts_list['price'] = gilts_list['Sequence'].apply(lambda x: price_df[x].loc[calculation_date])

X = ((gilts_list['Latest redemption date'] - pd.Timestamp("1970-01-01")).dt.days).to_list()
# X = [np.array(X)[:, np.newaxis]]
X = np.array(X).reshape(-1, 1)
y = gilts_list['price']

pipeline.fit(X, y)

gilts_list['decision_line'] = pipeline.predict(X)

condition = (gilts_list['decision_line'] - gilts_list['price']) > 2
gilts_list['segmentation'] = condition.apply(lambda x: 1 if x else 0)

In [ ]:
sns.set_theme(style="darkgrid")

# use the scatterplot function to build the bubble map
ax = sns.scatterplot(
    data=gilts_list,
    x="Latest redemption date",
    y="price",
    size="Amount_Outstanding",
    hue="segmentation",
    legend=False,
    sizes=(10, 500)
)

# show the graph
ax.plot(gilts_list['Latest redemption date'], gilts_list['decision_line'])
plt.show()

In [ ]:
curve = yield_curve_pw(calculation_date, gilts_list)

In [ ]:
f = plt.figure(figsize=(8, 5))
ax = f.add_subplot(1, 1, 1)


ax.xaxis.grid(True, "major", color="lightgray")
ax.yaxis.grid(True, "major", color="lightgray")
ax.xaxis.set_major_locator(YearLocator(10))

ax.yaxis.set_major_formatter(PercentFormatter(1.0))
styles = iter(["-", "--", ":", "-.","o","+"])

dates = [from_iso(calculation_date) + ql.Period(i, ql.Months) for i in range(12 * 48 + 1)]

day_count = ql.ActualActual(ql.ActualActual.Bond)

rates = [
curve.zeroRate(d, day_count, ql.Continuous).rate() for d in dates
]
ax.plot_date(
    [d.to_date() for d in dates],
    rates,
    next(styles),
)

In [ ]:
bond_yield = []
curve_yield = []
curve_price = []
first_bond = gilts_list['Latest redemption date'].min()
last_bond = gilts_list['Latest redemption date'].max()
for _, loo in gilts_list.iterrows():
    yld = bgs_gilt_yield(loo['Sequence'], calculation_date)
    bond_yield.append(yld)
    if (loo['Latest redemption date'] == first_bond) or (loo['Latest redemption date'] == last_bond):
        loo_yld = yld
        fitted_price = loo['price']
    else:
        exc_list = gilts_list.loc[~gilts_list['ISIN Code'].isin([loo['ISIN Code']])].copy()
        exc_curve = yield_curve_pw(calculation_date, exc_list)
        loo_mat = from_iso(loo['Latest redemption date'])
        loo_yld = exc_curve.zeroRate(loo_mat, day_count, ql.Continuous).rate()
        bond = Gilt(
            trade_date=from_iso(calculation_date),
            issue_date=from_iso(loo['Issue date']),
            maturity_date=from_iso(loo['Latest redemption date']),
            first_cpn_date=from_iso(loo['First coupon payable on date']),
            last_cpn_date=from_iso(loo['Latest redemption date']),
            coupon=loo['%']/100,
        )
        fitted_price = bond.price(loo_yld)
    curve_price.append(fitted_price)
    curve_yield.append(loo_yld)

gilts_list['curve_yield'] = curve_yield
gilts_list['curve_price'] = curve_price
gilts_list['yield'] = bond_yield

gilts_list['dislocation'] = (gilts_list['curve_yield'] - gilts_list['yield']).abs()
gilts_list['price_diff'] = (gilts_list['curve_price'] - gilts_list['price']).abs()


In [ ]:
gilts_list.sort_values('dislocation', ascending=False).head(10)

In [ ]:
gilts_list.sort_values('price_diff', ascending=False).head(10)

In [ ]:
focus = gilts_list.loc[(gilts_list['segmentation'] == 1)&(gilts_list['price']<100)].copy()

In [ ]:
focus['Amount_Outstanding'].sum()

In [ ]:
((100 - focus['price'])*focus['Amount_Outstanding']).sum()/100